In [1]:
import sys
sys.path.append('..')
import os
import sklearn.metrics.cluster
import sklearn.cluster
import seaborn as sns
import scanpy as sc
import sklearn as sk
from sklearn.neighbors import NearestNeighbors
import matplotlib as mpl
from matplotlib import pyplot as plt
import skdim
import seaborn as sns
import time
import numpy as np
import pandas as pd
import pickle
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
import rpy2.robjects.packages as rpackages
import utils
import anndata2ri
import gc
import benchmark_common_functions
from itertools import product

anndata2ri.activate()
ro.numpy2ri.activate()
readRDS = ro.r['readRDS']
%load_ext rpy2.ipython

In [3]:
rootdir = '../data/DuoClustering2018/sce_full/'
resdir = '../results/DuoClustering2018/sce_full/'

fnames = os.listdir(rootdir)
fnames = [f for f in fnames if 'sce' in f and '.rds' in f]

#get_res_path = lambda fname: f'{resdir}{fname}_norm{do_norm}_scale{norm_scale}_log{do_log}_pca{do_pca}_ncomps{n_comps}_{metric}_weighted{weighted}_knn{n_neighbors}_seed{seed}.pkl'
#get_res_path_orig = lambda fname: f'{resdir}{fname}_orig_pca{do_pca}_ncomps{n_comps}_{metric}_weighted{weighted}_knn{n_neighbors}_seed{seed}.pkl'

get_res_path = lambda fname: f'{resdir}{fname}_norm{do_norm}_scale{norm_scale}_log{do_log}_pca{do_pca}_ncomps{n_comps}_{metric}_weighted{weighted}_knn_sqrt_seed{seed}.pkl'
get_res_path_orig = lambda fname: f'{resdir}{fname}_orig_pca{do_pca}_ncomps{n_comps}_{metric}_weighted{weighted}_knn_sqrt_seed{seed}.pkl'
get_res_path_sc = lambda fname: f'{resdir}{fname}_norm{do_norm}_scale{norm_scale}_log{do_log}_pca{do_pca}_ncomps{n_comps}_{metric}_weighted{weighted}_knn_sqrt_seed{seed}_scanpyhubness.pkl'


# Run all datasets original

In [3]:
#fixed params
do_norm = False
norm_scale = False
do_log = False
do_pca = True
n_neighbors = 10
seed = 0

#vary params
metric=('cosine','euclidean')
n_comps=(25,50,100,500)
weighted=(True,)

params_list = list(product(metric,n_comps,weighted))

In [ ]:
for metric,n_comps,weighted in params_list:
    print(metric,n_comps,weighted)
    for fname in fnames:
        if get_res_path_orig(fname[:-4]).split('/')[-1] in os.listdir(resdir):
            continue
        print('\t\t'+fname)
        if fname in fnames[:2]+['sce_full_KumarTCC.rds']:
            adata = sc.read_csv(rootdir+fname.split('.')[0]+'_log.csv').transpose()
            adata.obs['phenoid'] = pd.read_csv('../data/DuoClustering2018/sce_full/'+fname.split('.')[0]+'_labels.csv').iloc[:,1].values

        else:
            adata = readRDS(rootdir+fname)
            adata.X = adata.layers['logcounts']

        results_dict = benchmark_common_functions.clustering_analysis(
            adata,
            true_labels = adata.obs['phenoid'],
            do_norm = do_norm,
            norm_scale = norm_scale,
            do_log = do_log,
            do_pca = do_pca,
            n_neighbors=int(round(np.sqrt(len(adata)))),
            n_clusters= adata.obs['phenoid'].nunique(),
            metric=metric,
            weighted=weighted,  #weighted adjmat for louvain/leiden clustering ?
            seed=seed,
            n_comps=n_comps
        )

        with open(get_res_path_orig(fname[:-4]),'wb') as f:
            pickle.dump(results_dict,f)
    
        %xdel adata
        gc.collect()

cosine 25 True
		sce_full_TrapnellTCC.rds


# Run all datasets

In [4]:
#fixed params
do_log = True
do_pca = True
n_neighbors = 10
seed = 0

#vary params
metric=('cosine','euclidean')
n_comps=(25,50,100,500)
weighted=(True,)
do_norm = ('duo','seurat')#,'zheng17')
norm_scale = (True,False)
params_list = list(product(metric,n_comps,weighted,do_norm,norm_scale))

In [6]:
for metric,n_comps,weighted,do_norm,norm_scale in params_list:
    print(metric,n_comps,weighted,do_norm,norm_scale)
    for fname in fnames:
        if get_res_path(fname[:-4]).split('/')[-1] in os.listdir(resdir):
            continue
        print('\t\t'+fname)
        if fname in fnames[:2]+['sce_full_KumarTCC.rds']:
            adata = sc.read_csv(rootdir+fname.split('.')[0]+'.csv').transpose()
            adata.obs['phenoid'] = pd.read_csv('../data/DuoClustering2018/sce_full/'+fname.split('.')[0]+'_labels.csv').iloc[:,1].values
        elif fname == 'sce_full_TrapnellTCC.rds':
            adata = sc.read_csv(rootdir+'sce_full_TrapnellTCC.csv').transpose()
            adata.obs['phenoid'] = readRDS('../data/DuoClustering2018/sce_filteredExpr10/sce_filteredExpr10_TrapnellTCC.rds').obs['phenoid']
        else:
            adata = readRDS(rootdir+fname)
            if 'Sim' in fname:
                adata.X = adata.layers['counts']
        results_dict = benchmark_common_functions.scanpy_hubness_analysis(
            adata,
            true_labels = adata.obs['phenoid'],
            do_norm = do_norm,
            norm_scale = norm_scale,
            do_log = do_log,
            do_pca = do_pca,
            n_neighbors=int(round(np.sqrt(len(adata)))),
            n_clusters= adata.obs['phenoid'].nunique(),
            metric=metric,
            weighted=weighted,  #weighted adjmat for louvain/leiden clustering ?
            seed=seed,
            n_comps=n_comps
        )

        with open(get_res_path(fname[:-4]),'wb') as f:
            pickle.dump(results_dict,f)
    
        %xdel adata
        %R rm(list = ls())
        %R gc()
        time.sleep(2)
        gc.collect()

cosine 25 True duo True
cosine 25 True duo False
cosine 25 True seurat True
cosine 25 True seurat False
		sce_full_KohTCC.rds
		seurat norm retained 54558 genes
			Scoring: 0.43 mn
		sce_full_Kumar.rds
		seurat norm retained 12108 genes
			Scoring: 0.02 mn
		sce_full_KumarTCC.rds
		seurat norm retained 45026 genes
			Scoring: 0.26 mn
		sce_full_SimKumar4easy.rds


R[write to console]: Loading required package: splatter



		seurat norm retained 11699 genes
			Scoring: 0.1 mn
		sce_full_SimKumar4hard.rds
		seurat norm retained 11627 genes
			Scoring: 0.08 mn
		sce_full_SimKumar8hard.rds
		seurat norm retained 11575 genes
			Scoring: 0.07 mn
		sce_full_Trapnell.rds
		seurat norm retained 9891 genes
			Scoring: 0.03 mn
		sce_full_TrapnellTCC.rds
		seurat norm retained 52445 genes
			Scoring: 0.17 mn
		sce_full_Zhengmix4eq.rds
		seurat norm retained 2874 genes
			Scoring: 0.16 mn
		sce_full_Zhengmix4uneq.rds
		seurat norm retained 2913 genes


ValueError: kth(=59) out of bounds (59)

In [5]:
for metric,n_comps,weighted,do_norm,norm_scale in params_list:
    print(metric,n_comps,weighted,do_norm,norm_scale)
    for fname in fnames:
        if get_res_path_sc(fname[:-4]).split('/')[-1] in os.listdir(resdir):
            continue
        print('\t\t'+fname)
        if fname in fnames[:2]+['sce_full_KumarTCC.rds']:
            adata = sc.read_csv(rootdir+fname.split('.')[0]+'.csv').transpose()
            adata.obs['phenoid'] = pd.read_csv('../data/DuoClustering2018/sce_full/'+fname.split('.')[0]+'_labels.csv').iloc[:,1].values
        elif fname == 'sce_full_TrapnellTCC.rds':
            adata = sc.read_csv(rootdir+'sce_full_TrapnellTCC.csv').transpose()
            adata.obs['phenoid'] = readRDS('../data/DuoClustering2018/sce_filteredExpr10/sce_filteredExpr10_TrapnellTCC.rds').obs['phenoid']
        else:
            adata = readRDS(rootdir+fname)
            if 'Sim' in fname:
                adata.X = adata.layers['counts']
        results_dict = benchmark_common_functions.scanpy_hubness_analysis(
            adata,
            do_norm = do_norm,
            norm_scale = norm_scale,
            do_log = do_log,
            do_pca = do_pca,
            n_neighbors=int(round(np.sqrt(len(adata)))),
            metric=metric,
            weighted=weighted,  #weighted adjmat for louvain/leiden clustering ?
            seed=seed,
            n_comps=n_comps
        )

        with open(get_res_path_sc(fname[:-4]),'wb') as f:
            pickle.dump(results_dict,f)
    
        %xdel adata
        %R rm(list = ls())
        %R gc()
        time.sleep(2)
        gc.collect()

cosine 25 True duo True
cosine 25 True duo False
cosine 25 True seurat True
cosine 25 True seurat False
cosine 50 True duo True
cosine 50 True duo False
cosine 50 True seurat True
cosine 50 True seurat False
cosine 100 True duo True
cosine 100 True duo False
cosine 100 True seurat True
cosine 100 True seurat False
cosine 500 True duo True
cosine 500 True duo False
cosine 500 True seurat True
cosine 500 True seurat False
euclidean 25 True duo True
euclidean 25 True duo False
euclidean 25 True seurat True
euclidean 25 True seurat False
euclidean 50 True duo True
euclidean 50 True duo False
euclidean 50 True seurat True
euclidean 50 True seurat False
euclidean 100 True duo True
euclidean 100 True duo False
euclidean 100 True seurat True
euclidean 100 True seurat False
euclidean 500 True duo True
euclidean 500 True duo False
		sce_full_SimKumar8hard.rds


R[write to console]: Loading required package: splatter



		duo norm retained 5000 genes
			Scoring: 0.12 mn
		sce_full_Trapnell.rds
		duo norm retained 5000 genes
			Scoring: 0.01 mn
		sce_full_TrapnellTCC.rds
		duo norm retained 5000 genes
			Scoring: 0.07 mn
		sce_full_Zhengmix4eq.rds
		duo norm retained 5000 genes
			Scoring: 0.55 mn
		sce_full_Zhengmix4uneq.rds
		duo norm retained 5000 genes
			Scoring: 0.97 mn
		sce_full_Zhengmix8eq.rds
		duo norm retained 5000 genes
			Scoring: 0.77 mn
euclidean 500 True seurat True
		sce_full_Koh.rds
		seurat norm retained 11419 genes
			Scoring: 0.1 mn
		sce_full_KohTCC.rds
		seurat norm retained 54558 genes
			Scoring: 0.64 mn
		sce_full_Kumar.rds
		seurat norm retained 12108 genes
			Scoring: 0.04 mn
		sce_full_KumarTCC.rds
		seurat norm retained 45026 genes
			Scoring: 0.21 mn
		sce_full_SimKumar4easy.rds
		seurat norm retained 11699 genes
			Scoring: 0.14 mn
		sce_full_SimKumar4hard.rds
		seurat norm retained 11627 genes
			Scoring: 0.17 mn
		sce_full_SimKumar8hard.rds
		seurat norm retained 1157